In [110]:
import pandas as pd
import numpy as np
import re

In [111]:
def ATOM_pdb_line(atomid, x, y, z, b, atomname = "C"):
    """
    atomid should already be a string-formatted integer < 10000
    x, y, z should already be in the format of :.3f
    b should already be in the format of :.2f 
    """
    end_atomid = 11
    start_atomname = 13
    end_resname = 20 # let us take resname same as atomname
    end_resid = 26 # let us take the resid same as the atomid
    end_x = 38
    end_y = 46
    end_z = 54
    end_b = 66

    out = [" "]*80 
    out.append("\n")
    out[0:3] = "ATOM"
    out[end_atomid - len(atomid): end_atomid] = atomid
    out[start_atomname: start_atomname + len(atomname)] = atomname
    out[end_resname - len(atomname): end_resname] = atomname
    out[end_resid - len(atomid): end_resid] = atomid
    out[end_x - len(x): end_x] = x
    out[end_y - len(y): end_y] = y
    out[end_z - len(z): end_z] = z
    out[end_b - len(b): end_b] = b
    
    return ''.join(out) 

def CONECT_pdb_line(starting_atomid, connected_atomids):
    """
    starting_atomid is the id of the atom from which connections start
    connected atomids is a list of at most 4 atomid of atoms connected to the starting atom

    it is allowed to have multiple CONECT lines for the same atom, if that atom is connected to more than 4 atoms
    """
    end_starting_atomid = 11
    end_connected_atomids = [16, 21, 26, 31]

    out = [" "]*80 
    out.append("\n")
    out[0:5] = "CONECT"
    out[end_starting_atomid - len(starting_atomid): end_starting_atomid] = starting_atomid
    for end, idi in zip(end_connected_atomids, connected_atomids):
        out[end - len(idi): end] = idi
    
    return ''.join(out) 

def CA_coordinates(pdb_path):
    """
    Note that in the case that i have more than one CA in the same residue, then the first CA is considered
    """
    
    rexp = re.compile('^ATOM[ 0-9]{9}CA[ 0-9a-zA-Z]{8}([ 0-9]{4})[ 0-9a-zA-Z]{4}([-. 0-9]{8})([-. 0-9]{8})([-. 0-9]{8})')
    
    CA_coords = []
    
    with open(pdb_path, "r") as file:
        current_res = None
        for l in file.readlines(): 
            m = rexp.match(l) 
            if m:
                if float(m.group(1)) != current_res:
                    CA_coords.append([float(m.group(1)), float(m.group(2)), float(m.group(3)), float(m.group(4))])
                    current_res = float(m.group(1))
                
    return CA_coords

def get_connections(matrix_path, treshold = None): 
    matrix = np.loadtxt(matrix_path, delimiter = " ")
    nresids = np.shape(matrix)[0]
    conn = []

    if treshold == None:
        treshold = 0.5

    for i in range(nresids):
        single_res_connections = []
        for j in range(nresids):
            if type(treshold) == list: 
                if (np.abs(matrix[i, j]) >= treshold[0]) and (np.abs(matrix[i, j]) <= treshold[1]):
                    single_res_connections.append(j)
            else:        
                if np.abs(matrix[i, j]) >= treshold:
                    single_res_connections.append(j)
        conn.append(single_res_connections)   
    return conn

In [112]:
path = "/home/giacomo/comp_bio/cbp_final/data/03_analyzed/graphs/ccs.pdb"
matrix_path = "/home/giacomo/comp_bio/cbp_final/data/03_analyzed/graphs/macro_IIN_unweighted.dat"
matrix_w_path = "/home/giacomo/comp_bio/cbp_final/data/03_analyzed/graphs/macro_IIN_weighted.dat"
path_pdb = "/home/giacomo/comp_bio/cbp_final/data/00_external/pdb_3EIG_prot.pdb"

path_graph_pdb = "/home/giacomo/comp_bio/cbp_final/data/03_analyzed/graphs/manual_graph_c07.pdb"

In [113]:
# get coordinater for each residue
CA_coords = CA_coordinates(path_pdb)

# Using the weighted matrix

In [114]:
conn_w = get_connections(matrix_w_path, [0.7, 0.8])

In [115]:
print(conn_w)

[[], [], [], [130], [112], [120], [112, 114], [134], [], [13], [12], [], [10], [9], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [164], [114], [], [], [], [], [], [], [], [], [], [], [50], [], [48], [], [], [], [], [], [82], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [82], [], [], [], [56, 78], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [108], [], [], [], [104], [], [], [], [4, 6], [119], [6, 37], [], [], [], [], [113], [5], [], [], [], [147], [], [], [], [], [], [3], [], [], [], [7], [], [], [], [], [], [], [], [], [], [], [], [], [124], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [36], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [116]:
with open(path_graph_pdb, "w") as file: 
    # first one line for each residue with at least 2 connections and the beta factor as the number of connections
    for coords, con in zip(CA_coords, conn_w): 
        #if len(con) >= 2:
        line = ATOM_pdb_line(f"{int(coords[0]):d}", f"{coords[1]:.3f}", f"{coords[2]:.3f}", f"{coords[3]:.3f}", f"{len(con):.2f}")
        file.write(line)
    # then connection lines
    for i, con in enumerate(conn_w):
        if len(con) > 0:
            tocon = [f"{j}" for j in con]
            line = CONECT_pdb_line(f"{i+1}", tocon)
            file.write(line)
    # in the end END
    file.write("END")